# Car Price Prediction

In [1]:
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import seaborn as sns
from bs4 import BeautifulSoup as bs
import requests
import re
import time
from datetime import date
import numpy as np
import pickle
from uszipcode import SearchEngine
%matplotlib inline

/Users/darren/opt/anaconda3/envs/regression/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Getting scraped data from csv file

In [2]:
df = pd.read_csv('data/car_info.csv', index_col=0)
df2 = pd.read_csv('data/car_info_extended.csv', index_col=0)
df = pd.concat([df, df2])
df = df.dropna()
df = df[df.zipcode != 0]
# train_df, test_df = train_test_split(df, test_size=0.2)
# train_df.to_csv('data/train_df.csv', encoding='utf-8')
# test_df.to_csv('data/test_df.csv', encoding='utf-8')

In [3]:
# train_df = pd.read_csv('data/train_df.csv', index_col=0)
# test_df = pd.read_csv('data/test_df.csv', index_col=0)

In [4]:
# train_df.head()

## Data cleaning

### Fuel type

In [5]:
def is_gas(row):
    regex1 = re.compile('.*Gas.*')
    regex2 = re.compile('.*Flex.*')
    if regex1.match(row.fuel_type) or \
    regex2.match(row.fuel_type) or \
    row.fuel_type == 'Hybrid ':
        return 1
    else:
        return 0
    
def is_diesel(row):
    regex1 = re.compile('Diesel.*')
    regex2 = re.compile('.*Flex Fuel.*')
    if regex1.match(row.fuel_type) or \
    regex2.match(row.fuel_type):
        return 1
    else:
        return 0
    
def is_electric(row):
    regex = re.compile('.*Electric.*')
    if regex.match(row.fuel_type) or \
    row.fuel_type == 'Hybrid ':
        return 1
    else:
        return 0

In [6]:
df['is_gas'] = df.apply(lambda row : is_gas(row), axis=1)
df['is_diesel'] = df.apply(lambda row : is_diesel(row), axis=1)
df['is_electric'] = df.apply(lambda row : is_electric(row), axis=1)

### Year & make

In [7]:
def get_year_make_model(row):
    i = row['name']
    w = i.split()
    return w

In [8]:
df['age'] = df.apply(lambda row : date.today().year - int(get_year_make_model(row)[0]), axis=1)\
    .astype('int')
df['make'] = df.apply(lambda row : get_year_make_model(row)[1], axis=1)
# df['model'] = df.apply(lambda row : get_year_make_model(row)[2], axis=1)

### MPG

In [9]:
# drop rows with blank mpg
df = df[df['mpg'] != '–']
df = df[df['mpg'] != 'None']

In [10]:
def get_low_mpg(row):
    if row.mpg == '–' or row.mpg == 'None':
        return 25.4
    else:
        i = row.mpg.find('–')
        if i != -1:
            return row.mpg[0:i]
        else:
            return row.mpg

def get_high_mpg(row):
    if row.mpg == '-' or row.mpg == 'None':
        return 25.4
    else:
        i = row.mpg.find('–')
        if i != -1:
            return row.mpg[i+1:]
        else:
            return row.mpg

In [11]:
df['mpg'] = df.apply(lambda row : int(get_low_mpg(row)) + int(get_high_mpg(row)), axis=1).astype('int')

### Transmission

In [12]:
def is_auto(row):
    regex = re.compile('.*([A|a]uto|A/T|Dual Shift).*')
    if regex.match(row.transmission):
        return 1
    else:
        return 0

def is_cvt(row):
    regex = re.compile('.*(CVT|Variable).*')
    if regex.match(row.transmission):
        return 1
    else:
        return 0
    
def is_manual(row):
    regex = re.compile('.*(Manual|M/T).*')
    if regex.match(row.transmission):
        return 1
    else:
        return 0
    
def get_speed(row):
    tmp = row.transmission
    tmp = re.sub('[^0-9]', '', tmp)
    if tmp == '':
        return 6
    else:
        return int(tmp)

In [13]:
df['is_auto'] = df.apply(lambda row : is_auto(row), axis=1)
df['is_cvt'] = df.apply(lambda row : is_cvt(row), axis=1)
df['is_manual'] = df.apply(lambda row : is_manual(row), axis=1)
df['speed'] = df.apply(lambda row : get_speed(row), axis=1)

### Engine

In [14]:
def get_L(row):
    i = row.engine.find('L')
    if i == -1:
        return 0
    else:
        return float(row.engine[i-3:i])

def get_V(row):
    i = row.engine
    a = re.compile('.*16V.*')
    b = re.compile('.*24V.*')
    c = re.compile('.*32V.*')
    d = re.compile('.*48V.*')
    e = re.compile('.*12V.*')
    if a.match(i):
        return '16V'
    elif b.match(i):
        return '24V'
    elif c.match(i):
        return '32V'
    elif d.match(i):
        return '48V'
    elif e.match(i):
        return '12V'
    else:
        return 'Others'
    
def is_PDI(row):
    engine = row.engine
    a = re.compile('.*PDI.*')
    if a.match(engine):
        return 1
    else:
        return 0
    
def is_GDI(row):
    engine = row.engine
    a = re.compile('.*GDI.*')
    if a.match(engine):
        return 1
    else:
        return 0

def is_MPFI(row):
    engine = row.engine
    a = re.compile('.*MPFI.*')
    if a.match(engine):
        return 1
    else:
        return 0
    
def is_DOHC(row):
    engine = row.engine
    a = re.compile('.*DOHC.*')
    if a.match(engine):
        return 1
    else:
        return 0
    
def is_cylinder_engine(row):
    engine = row.engine
    a = re.compile('.*Cylinder Engine.*')
    if a.match(engine):
        return 1
    else:
        return 0

In [15]:
df['engine_L'] = df.apply(lambda row : get_L(row), axis=1).astype('float')
df['engine_V'] = df.apply(lambda row : get_V(row), axis=1)
df['is_PDI'] = df.apply(lambda row : is_PDI(row), axis=1).astype('int')
df['is_GDI'] = df.apply(lambda row : is_GDI(row), axis=1).astype('int')
df['is_MPFI'] = df.apply(lambda row : is_MPFI(row), axis=1).astype('int')
df['is_DOHC'] = df.apply(lambda row : is_DOHC(row), axis=1).astype('int')
df = pd.get_dummies(df, columns=['engine_V'])

### Drivetrain

In [16]:
# drop rows with blank drivetrain
df.drivetrain = df.drivetrain.apply(lambda x : x[1:-1])
df = df[df['drivetrain'] != '–']

In [17]:
def get_drivetrain(row):
    i = row.drivetrain
    four = re.compile('.*(Front|Rear|FWD).*')
    two = re.compile('.*(All|Four|4WD|AWD).*')
    if four.match(i):
        return '4'
    elif two.match(i):
        return '2'
    else:
        return 'Others'

In [18]:
df['drivetrain'] = df.apply(lambda row : get_drivetrain(row), axis=1)
df = pd.get_dummies(df, columns=['drivetrain'])

## Zipcode

In [19]:
zipcode_to_crime_rate = pd.DataFrame(columns=['zipcode', 'crime_rate'])
zipcode_to_crime_rate['zipcode'] = df.zipcode.unique()
zipcode_to_crime_rate.crime_rate =\
    zipcode_to_crime_rate.crime_rate.apply(lambda x : 0)

In [20]:
# get zip code
def get_population(row):
    engine = SearchEngine()
    zipcode = engine.by_zipcode(row.zipcode)
    return zipcode.population

In [21]:
df['population'] = df.apply(lambda row : get_population(row), axis=1)

## Car make and model

In [22]:
# df = pd.get_dummies(df, columns=['make_model'], \
#                     prefix='mm', prefix_sep='_')

In [23]:
def get_make(row):
    make = df.make.value_counts()
    make = make[make>100].index.tolist()
    if row.make in make:
        return row.make
    else:
        return 'Others'

In [24]:
# df['make'] = df.apply(lambda row : get_make(row), axis=1)

In [25]:
df = pd.get_dummies(df, columns=['make'])

### Drop useless columns

In [26]:
# sns.pairplot(df[['price', 'mileage', 'age', 'population', 'mpg', 'speed']])

In [27]:
df = df.drop(columns=\
             ['name', 'mpg', 'speed', 'fuel_type', 'transmission', 'engine', 'personal_use', 'zipcode'])
df = df.dropna()

In [40]:
# train_df, test_df = train_test_split(df, test_size=0.2)
# train_df.to_csv('data/train_df.csv', encoding='utf-8')
# test_df.to_csv('data/test_df.csv', encoding='utf-8')

In [29]:
df.head()

,price,mileage,oneowner,is_gas,is_diesel,is_electric,age,is_auto,is_cvt,is_manual,...,make_Rolls-Royce,make_Saturn,make_Scion,make_Subaru,make_Suzuki,make_Tesla,make_Toyota,make_Volkswagen,make_Volvo,make_smart
0,54891.0,35283,3,1,0,0,3,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,28803.0,66508,3,1,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,27500.0,75651,0,1,0,0,5,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,36000.0,48800,0,1,0,0,4,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,79989.0,13727,3,1,0,0,5,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Building regression model

### Lasso Regression model

In [32]:
train_df, test_df = train_test_split(df, test_size=0.2)
X = train_df.drop(columns=['price'], axis=1)
y = train_df.price

In [41]:
train_df.columns

Index(['price', 'mileage', 'oneowner', 'is_gas', 'is_diesel', 'is_electric',
       'age', 'is_auto', 'is_cvt', 'is_manual', 'engine_L', 'is_PDI', 'is_GDI',
       'is_MPFI', 'is_DOHC', 'engine_V_12V', 'engine_V_16V', 'engine_V_24V',
       'engine_V_32V', 'engine_V_48V', 'engine_V_Others', 'drivetrain_2',
       'drivetrain_4', 'drivetrain_Others', 'population', 'make_Acura',
       'make_Alfa', 'make_Aston', 'make_Audi', 'make_BMW', 'make_Bentley',
       'make_Buick', 'make_Cadillac', 'make_Chevrolet', 'make_Chrysler',
       'make_Dodge', 'make_Ferrari', 'make_Ford', 'make_GMC', 'make_Genesis',
       'make_Honda', 'make_Hyundai', 'make_INFINITI', 'make_Jaguar',
       'make_Jeep', 'make_Kia', 'make_Lamborghini', 'make_Land', 'make_Lexus',
       'make_Lincoln', 'make_Lotus', 'make_MINI', 'make_Maserati',
       'make_Mazda', 'make_McLaren', 'make_Mercedes-Benz', 'make_Mercury',
       'make_Mitsubishi', 'make_Nissan', 'make_Plymouth', 'make_Pontiac',
       'make_Porsche', 'make_R

In [35]:
# ridge regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
poly = PolynomialFeatures(degree=2)
X_scaled_poly = poly.fit_transform(X_scaled)
alphas = 10**np.linspace(-4, 4, 50)
ridge = RidgeCV(alphas=alphas, cv=5)
ridge.fit(X_scaled_poly, y)
ridge.score(X_scaled_poly, y)

0.8230144127265071

In [36]:
ridge.alpha_

494.1713361323828

In [37]:
X_test = test_df.drop(columns=['price'], axis=1)
y_test = test_df.price

X_test_scaled = scaler.transform(X_test)
X_test_scaled_poly = poly.transform(X_test_scaled)

ridge.score(X_test_scaled_poly, y_test)

0.7841333431258171

In [38]:
# save the model
with open('models/ridge_model.pkl', 'wb') as file:
    pickle.dump(ridge, file)

In [39]:
with open('models/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)